# Tutorial - Green Urban Scenarious
A demo of a minimal flow of GUS process.

For more advanced use cases, please see upcoming article. For more structured experiment and scenario analyis plese see model sepcification and code documentation within docs/ folder. 

In [ ]:
# Install the GUS framework.

import sys
!{sys.executable} -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple gus==0.1.7

In [1]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 3
%matplotlib inline
from pkg_resources import resource_filename
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os, sys, json
import pkgutil
import pprint
pp = pprint.PrettyPrinter(indent=1)
sns.set_theme()

## 0. Importing GUS modules

In [2]:
from src.gus.models import Urban
from src.gus.utilities import get_raster_data

## 1. Initilisation

### 1.1 Importing data on site, tree populations, experiment parameters

In [9]:
import logging
logfile = os.path.join("src", "gus/outputs/gus.log")
print(logfile)
logging.basicConfig(filename=logfile, format='%(asctime)s %(levelname)s %(message)s', level=logging.INFO)

# File that holds site/city other geolocated data and parameters
fname_config_site = resource_filename("src", "gus/inputs/site.json")

# File that include tree species specific parameters
fname_config_allometrics = resource_filename("src", "gus/inputs/allometrics.json")

# File that include individual tree data including their location on a 2D raster of the actual location
fname_data_trees = resource_filename("src", "gus/inputs/trees.csv")

# File that holds experiment parameters
fname_scenario_params = resource_filename("src", "gus/inputs/scenario.json")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ozzibroccoli/.pyenv/versions/3.8.0/envs/gus-test/lib/python3.8/site-packages/src/gus/outputs/gus.log'

In [7]:
f = open(fname_config_allometrics)
config_allometrics = json.loads(f.read())
#pp.pprint(config_allometrics)

In [8]:
f = open(fname_config_site)
config_site = json.loads(f.read())
pp.pprint(config_site)

{'area_impervious_in_m2': 500,
 'area_pervious_in_m2': 500,
 'area_total_in_m2': 1000,
 'area_tree_density_per_hectare': 400,
 'project_site_type': 'park',
 'weather': {'growth_season_mean': 200, 'growth_season_var': 7}}


In [ ]:
df_pop = pd.read_csv(fname_data_trees)
df_pop.tail()

: 

In [ ]:
f = open(fname_scenario_params)
scenario_params = json.loads(f.read())
pp.pprint(scenario_params)

: 

### 1.2 Creating a digital twin of the site

In [ ]:
site = Urban(df_pop,
            ff_config_allometrics,
            ff_config_site,
            scenario_params)

: 

In [ ]:
initial_locations = get_raster_data(site, counts=True)
initial_biomass = get_raster_data(site, var='biomass')

sns.set_style("ticks")
fig = plt.figure(figsize=(16,12))
plt.subplot(121)
plt.imshow(initial_locations, cmap='Greens')
plt.title("Tree Locations")
plt.subplot(122)
plt.title("Biomass Concentrations")
plt.imshow(initial_biomass, cmap='Blues')
#plt.colorbar(label = 'Biomass - Kg')
plt.show()

: 

## 2. Running the scenario

In [ ]:
for i in range(scenario_params['time_horizon_years']):
    site.step()

: 

## 3. Analysis

### 3.1 Individual tree level inspections 

In [ ]:
# Tree level individual variables
fname_output_trees = resource_filename("src", "gus/outputs/trees_yearly.json")
df_out_tree = site.datacollector.get_agent_vars_dataframe()
df_out_tree.to_csv(fname_output_trees, index=True)
df_out_tree.tail()

: 

In [ ]:
Specific_Tree = df_out_tree.xs(42, level="AgentID")
Specific_Tree.tail()

: 

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(16,8))
plt.subplot(221)
Specific_Tree.height.plot()
plt.xlabel('Years')
plt.ylabel('m')
plt.legend()
plt.subplot(222)
Specific_Tree.dbh.plot()
plt.xlabel('Years')
plt.ylabel('cm')
plt.legend()
plt.subplot(223)
Specific_Tree.carbon.plot()
plt.xlabel('Years')
plt.ylabel('Kg')
plt.legend()
plt.subplot(224)
Specific_Tree.dieback.plot()
plt.xlabel('Years')
plt.ylabel('ratio')
plt.legend()
plt.show()

: 

### 3.2 Site level inspections

In [ ]:
# Site level aggregate variables
df_out_site = site.datacollector.get_model_vars_dataframe()
df_out_site.tail()

: 

In [ ]:
fig = plt.figure(figsize=(16,8))
df_out_site.Seq.plot()
df_out_site.Released.plot()
plt.xlabel('Years')
plt.ylabel('Kg')
plt.legend()
plt.show()

: 

In [ ]:
locations = get_raster_data(site, counts=True)
biomass = get_raster_data(site, var='biomass')

sns.set_style("ticks")
fig = plt.figure(figsize=(16,16))
plt.subplot(221)
plt.imshow(initial_locations, cmap='Greens')
plt.title("Initial Tree Canopy")

plt.subplot(222)
plt.imshow(locations, cmap='Greens')
plt.title("Current Tree Canopy")

plt.subplot(223)
plt.title("Initial biomass concentrations")
plt.imshow(initial_biomass, cmap='Blues', vmin=0, vmax=biomass.max())
plt.colorbar(label = 'Biomass - Kg')

plt.subplot(224)
plt.title("Current biomass concentrations")
plt.imshow(biomass, cmap='Blues')
plt.colorbar(label = 'Biomass - Kg')
plt.show()

: 